# Assignment 1: Uninformed Search

Nathan Kluth

## Overview

For this assignment I have implemented Breadth-first and depth-first search.

Breadth-first and depth-first are two algorithms for performing
uninformed search---a search that does not use
knowledge about the goal of the search.  Below is an implementation of both
search algorithms in python.

## Included Code

In this jupyter notebook, I have implemented the following functions:

  * `breadthFirstSearch(startState, goalState, successorsf)` 
  * `depthFirstSearch(startState, goalState, successorsf)`
  
Each receives as arguments the starting state, the goal state, and a successors function.  `breadthFirstSearch` returns the breadth-first solution path as a list of states starting with the `startState` and ending with the `goalState`.  `depthFirstSearch` returns the depth-first solution path.

The functions implement the search algorithm as specified in [A3 Problem-Solving Agents](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/03 Problem-Solving Agents.ipynb) lecture notes. Each function wraps the main search function, which implements the algorithm.

# Examples

Here is a simple example.  States are defined by lower case letters.  A dictionary stores a list of successor states for each state in the graph that has successors.

In [248]:
successors = {'a':  ['b', 'c', 'd'],
              'b':  ['e', 'f', 'g'],
              'c':  ['a', 'h', 'i'],
              'd':  ['j', 'z'],
              'e':  ['k', 'l'],
              'g':  ['m'],
              'k':  ['z']}
successors

{'a': ['b', 'c', 'd'],
 'b': ['e', 'f', 'g'],
 'c': ['a', 'h', 'i'],
 'd': ['j', 'z'],
 'e': ['k', 'l'],
 'g': ['m'],
 'k': ['z']}

## Setup

The search functions accept a sucessor function which allows them to be used on other puzzles. For the simple graph search I will use the function below

In [249]:
import copy

def successorsf(state):
    """simple successors function that returns the list at the given value"""
    return copy.copy(successors.get(state, []))

In [250]:
successorsf('e') # grab the values at position e

['k', 'l']

# Function

In [251]:
def search(startState, goalState, successorsf, breadthFirst):
    """ runs either breadthFirst or depthFirset search depending on breadthFirst
    boolean provided as last argument. this search function is called by the lower
    breadthFirst or depthFirst functions
    
    startState is the first node to search from
    goalState is the desired destination node
    successorsf is action performed at each step on the children nodes
    breadthFirst is a boolean that determines subsequent runs
    """
    
    #initial states
    unExpanded = [(startState, None)]
    expanded = {}
    
    # save time by returning early if goalState is startState
    if (startState == goalState):
        return [startState]
    
    # used to check if item is in unExpanded list arleady and should be removed
    def exists(child):
        for item in unExpanded:
            if(item[0] == child):
                return True
        return False
    
    # assign parent to child
    def makePair(child):
        return (child, state)

    
    # search loop
    while (unExpanded):
        (state, parent) = unExpanded.pop() # pop from end of unexpanded
        children = successorsf(state) # find children of state
        
        expanded[state] = parent; # add state to expanded list

        # remove children that have already been expanded or found to prevent infinite loop
        for child in children[:]:
            if (exists(child) or expanded.get(child) or startState == child):                
                children.remove(child)

        # success case
        if (goalState in children):
            path = [state, goalState]
            while parent:
                path = [parent] + path
                parent = expanded[parent]

            return path

        children.sort(reverse=True) # sort for grading (ensure same results)
        modifiedChildren = list(map(makePair, children)) # make new list of children with parent
        # create new list to work through
        if (breadthFirst): 
            unExpanded = modifiedChildren + unExpanded 
        else: 
            unExpanded = unExpanded + modifiedChildren

### Breadth-First search

In [252]:
def breadthFirstSearch(startState, goalState, successorsf):
    """convenience wrapper for search function described above
    Follows breadthFirstSearch to find path from startState to goalState.
    Breadth-first search completely explores each level of the search 
    space before proceeding to the next."""
    
    return search(startState, goalState, successorsf, True)
        

In [253]:
print('Breadth-first')
print('path from a to a is', breadthFirstSearch('a', 'a', successorsf))
print('path from a to m is', breadthFirstSearch('a', 'm', successorsf))
print('path from a to z is', breadthFirstSearch('a', 'z', successorsf))

Breadth-first
path from a to a is ['a']
path from a to m is ['a', 'b', 'g', 'm']
path from a to z is ['a', 'd', 'z']


### Depth-first search

In [254]:
def depthFirstSearch(startState, goalState, successorsf):
    """convenience wrapper for search function described above
    Follows depthFirstSearch to find path from startState to goalState.
    Depth-first search completely explores a path until it ends, 
    then backs up a level and tries again."""
    
    return search(startState, goalState, successorsf, False)

In [255]:
print('Depth-first')
print('path from a to a is', depthFirstSearch('a', 'a', successorsf))
print('path from a to m is', depthFirstSearch('a', 'm', successorsf))
print('path from a to z is', depthFirstSearch('a', 'z', successorsf))

Depth-first
path from a to a is ['a']
path from a to m is ['a', 'b', 'g', 'm']
path from a to z is ['a', 'b', 'e', 'k', 'z']


# Other puzzles

The following code illustrates one possible state representation and shows results of a breadth-first and a dept-first search.  The example search uses a new camelSuccessorsf that shows how breadthFirstSearch and depthFirstSearch can use this callback to achieve different results. This means that the two search methods can be used on other puzzles! The rules for camelSuccessorsf are described [here](http://www.folj.com/puzzles/).

In [256]:
def camelSuccessorsf(state):
    """expects a tuple of R and L camels along with a space in between
    eg ('R', 'R', 'R', 'R', ' ', 'L', 'L', 'L', 'L')
    returns the next states possible - if camel has room to move
    it can move forward, if a camel can climb over it can move to the 2nd space
    """
    
    succs = []
    for index, camel in enumerate(state): # there is probably a better way to do this in python
        # camel has space to move
        if (camel == 'R' and len(state) > index+1 and state[index+1] == ' '):
            newState = list(state)
            newState[index+1] = 'R'
            newState[index] = ' '
            succs.append(tuple(newState))
        if (camel == 'L' and index - 1 > -1 and state[index-1] == ' '):
            newState = list(state)
            newState[index-1] = 'L'
            newState[index] = ' '
            succs.append(tuple(newState))
            
        # camel has space to climb over
        if (camel == 'R' and len(state) > index+2 and state[index+1] == 'L' and state[index+2] == ' '):
            newState = list(state)
            newState[index] = ' '
            newState[index+2] = 'R'
            succs.append(tuple(newState))
        if (camel == 'L' and index - 2 > -1 and state[index-1] == 'R' and state[index-2] == ' '):
            newState = list(state)
            newState[index] = ' '
            newState[index-2] = 'L'
            succs.append(tuple(newState))
            
    
    return succs

In [257]:
camelStartState = ('R', 'R', 'R', 'R', ' ', 'L', 'L', 'L', 'L')

In [258]:
camelGoalState = ('L', 'L', 'L', 'L', ' ', 'R', 'R', 'R', 'R')

In [259]:
camelSuccessorsf(camelStartState)

[('R', 'R', 'R', ' ', 'R', 'L', 'L', 'L', 'L'),
 ('R', 'R', 'R', 'R', 'L', ' ', 'L', 'L', 'L')]

In [260]:
children = camelSuccessorsf(camelStartState)
print(children[0])
camelSuccessorsf(children[0])

('R', 'R', 'R', ' ', 'R', 'L', 'L', 'L', 'L')


[('R', 'R', ' ', 'R', 'R', 'L', 'L', 'L', 'L'),
 ('R', 'R', 'R', 'L', 'R', ' ', 'L', 'L', 'L')]

In [261]:
bfs = breadthFirstSearch(camelStartState, camelGoalState, camelSuccessorsf)
print('Breadth-first solution: (', len(bfs), 'steps)')
for s in bfs:
    print(s)

dfs = depthFirstSearch(camelStartState, camelGoalState, camelSuccessorsf)
print('Depth-first solution: (', len(dfs), 'steps)')
for s in dfs:
    print(s)

Breadth-first solution: ( 25 steps)
('R', 'R', 'R', 'R', ' ', 'L', 'L', 'L', 'L')
('R', 'R', 'R', ' ', 'R', 'L', 'L', 'L', 'L')
('R', 'R', 'R', 'L', 'R', ' ', 'L', 'L', 'L')
('R', 'R', 'R', 'L', 'R', 'L', ' ', 'L', 'L')
('R', 'R', 'R', 'L', ' ', 'L', 'R', 'L', 'L')
('R', 'R', ' ', 'L', 'R', 'L', 'R', 'L', 'L')
('R', ' ', 'R', 'L', 'R', 'L', 'R', 'L', 'L')
('R', 'L', 'R', ' ', 'R', 'L', 'R', 'L', 'L')
('R', 'L', 'R', 'L', 'R', ' ', 'R', 'L', 'L')
('R', 'L', 'R', 'L', 'R', 'L', 'R', ' ', 'L')
('R', 'L', 'R', 'L', 'R', 'L', 'R', 'L', ' ')
('R', 'L', 'R', 'L', 'R', 'L', ' ', 'L', 'R')
('R', 'L', 'R', 'L', ' ', 'L', 'R', 'L', 'R')
('R', 'L', ' ', 'L', 'R', 'L', 'R', 'L', 'R')
(' ', 'L', 'R', 'L', 'R', 'L', 'R', 'L', 'R')
('L', ' ', 'R', 'L', 'R', 'L', 'R', 'L', 'R')
('L', 'L', 'R', ' ', 'R', 'L', 'R', 'L', 'R')
('L', 'L', 'R', 'L', 'R', ' ', 'R', 'L', 'R')
('L', 'L', 'R', 'L', 'R', 'L', 'R', ' ', 'R')
('L', 'L', 'R', 'L', 'R', 'L', ' ', 'R', 'R')
('L', 'L', 'R', 'L', ' ', 'L', 'R', 'R', 'R'